# HOW CAN THE PROBLEM OF WORLD FOOD SHORTAGE BE SOLVED USING AFRICA AS A CASE STUDY?

In [28]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [29]:
#import datasets
prod=pd.read_csv('Africa Food Production (2004 - 2013).csv')
supply=pd.read_csv('Africa Food Supply (2004 - 2013).csv')

In [30]:
#pd.set_option("display.max_rows", None)
p_=prod.groupby(["Country", "Year"])["Value"].sum()
p_=pd.DataFrame(p_)
prod=p_.reset_index()
#prod



In [31]:
#total amount of food produced vs consumed from 2004 - 2013
p_sum = prod["Value"].sum()
print("The total food produced from 2004-2013 is ",p_sum)

s_sum = supply["Value"].sum()
print("The total food consumed from 2004-2013 is ",s_sum)


The total food produced from 2004-2013 is  7575116
The total food consumed from 2004-2013 is  1111399


In [32]:
colors = ['green',] * 2
colors[1]="red"
x=['FOOD PRODUCED', 'FOOD CONSUMED']
y= [p_sum,s_sum]

fig = go.Figure(data=[go.Bar(x=x,
                        y=y,
                        marker_color=colors,
                        text=y,
                        textposition="auto",
                        
                            textsrc="")
                        ])
fig.update_layout(title_text='FOOD PRODUCED VS FOOD CONSUMED FROM 2004-2013')
fig.show()

FROM THE ABOVE COMPARISON, WE CAN SEE THAT OVER A SPAN OF 10 YEARS, AFRICA 
PRODUCED OVER 7M ITEMS OF FOOD, WHILE ONLY ABOUT 1M ITEMS WERE CONSUMED BY THE TOTAL POPULATION.

LETS OBSERVE THE PRODUCTION AND SUPPLY TRENDS TO SEE COUNTRIES THAT PRODUCE AND CONSUME MORE FOOD AND THE COUNTRIES THAT PRODUCE BELOW AVERAGE FOOD ITEMS OVER A PERIOD OF 10 YRS FROM 2004-2013.

In [33]:
prod_av = pd.DataFrame(prod.groupby(["Year"])["Value"].mean())
prod_av = prod_av.rename({"Value":"Prod_value"}, axis=1)
#prod_av
supply_av = pd.DataFrame(supply.groupby(["Year"])["Value"].mean())
supply_av = supply_av.rename({"Value":"Supply_value"}, axis=1)
#supply_av
prod_supply_av = pd.concat([prod_av, supply_av], axis=1)
prod_supply_av.reset_index(level=0, inplace=True)
#prod_supply_av

In [34]:
years = prod_supply_av.Year

fig = go.Figure()
fig.add_trace(go.Bar(
    x=years,
    y=prod_supply_av.Prod_value,
    name='Total food produced',
    marker_color='green'
))
fig.add_trace(go.Bar(
    x=years,
    y=prod_supply_av.Supply_value,
    name='total food consumed',
    marker_color='red'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45, 
                 title_text="AVERAGE FOOD PRODUCED VS CONSUMED FOR EACH YEAR.")
fig.show()


FROM THE ABOVE CHART, WE CAN SEE THAT THE PRODUCTION INCREASES STEADILY AND FAR EXCEEDS THE CONSUMPTION WHICH APPEARS RATHER FIXED WITH MINIMAL CHANGE. 

NOW, LET US CONSIDER THE MEAN, MEDIAN, OUTLIERS, AND QUARTILES FOR FOOD PRODUCED FROM 2004-2013

In [35]:
dt=p_
df = px.data.tips()
fig=px.box(df, y=dt['Value'])
fig.update_traces(quartilemethod="linear",notched=True , boxmean=True)
fig.update_layout(title="AVERAGE AND MEDIAN FOOD PRODUCTION, OUTLIERS, QUARTILES AND IQR RANGES")

fig.show()

FROM THE ABOVE BOXPLOT, A QUICK HOVER REVEALS THE FOLLOWING INFORMATION:
    1. MANY OUTLIERS ARE PRESENT
    2. MEAN : 16.83K(2dp)
    3. MEDIAN : 7.88K(2dp)
    4. Q1 : 2.29K(2dp)
    5. Q3 : 20.73K(2dp)
    
FROM THIS OBSERVATION, IT IS OBVIOUS THAT MOST AFRICAN COUNTRIES PRODUCE FOOD THAT LIES BELOW THE AVERAGE (AT ABOUT 17KT) OVER THE PERIOD OF 10 YRS. HOWEVER, A FEW COUNTRIES PRODUCE WELL ABOVE AVERAGE.

EMPLOYING A CHOROPLETH MAP, WE CAN EASILY SEE AL THE COUNTRIES WITH THE OUTLIERS (COUNTRIES PRODUCING ALOT MORE FOOT ITEMS)

In [36]:
pop = pd.read_csv("API_SP.POP.TOTL_DS2_en_csv_v2_1308146.csv", skiprows=3)

In [37]:
pop=pop.replace("Congo, Dem. Rep.", "Congo")
pop=pop.replace("Egypt, Arab Rep.", "Egypt")
pop=pop.replace("Gambia, The", "Gambia")
pop=pop.replace("Eswatini", "Swaziland")
pop=pop.replace("Tanzania", "United Republic of Tanzania")

In [38]:
C = pd.DataFrame(pop["Country Name"])
C = C.rename(columns={"Country Name":"Country"})
#C.head()

In [39]:
CC = pd.DataFrame(pop["Country Code"])
CC.head()
#I will set the "Country Name" to index, to enable column merge/join
CC= CC.join(other=C)
CC=CC.set_index("Country")
#CC

In [40]:
import plotly.express as px

prod_map = pd.DataFrame(prod.groupby(["Country","Year"])["Value"].mean()).reset_index()
prod_map = prod_map.set_index("Country")
prod_map = prod_map.join(other=CC).reset_index()
prod_map

fig = px.choropleth(prod_map,
                    locations='Country Code',
                    locationmode="ISO-3",
                    hover_name="Country",
                    color=prod_map['Value'],
                    scope="africa",
                    color_continuous_scale=px.colors.sequential.Teal,
                    animation_frame="Year",
                 )
fig.update_layout(title="MEAN FOOD PRODUCTION IN AFRICA FROM 2004-2013")  
fig.show()

FROM THE FIGURE ABOVE, WE SEE THAT THE TOP FOOD PRODUCING COUNTRY IS:
    1. NIGERIA AT 179.631KT

AND THE LEAST FOOD PRODUCING COUNTRY IS:
    1. DJIBOUTI AT JUST 76KT
 

ALSO FOR THE SUPPLY TREND OVER TIME, WE HAVE:

In [41]:
import plotly.express as px

supply_map = pd.DataFrame(supply.groupby(["Country","Year"])["Value"].mean()).reset_index()
supply_map = supply_map.set_index("Country")
supply_map = supply_map.join(other=CC).reset_index()
supply_map

fig = px.choropleth(supply_map,
                    locations='Country Code',
                    locationmode="ISO-3",
                    hover_name="Country",
                    color=supply_map['Value'],
                    scope="africa",
                    #colorscale="tealgrn",
                    color_continuous_scale=px.colors.sequential.Magenta,

                    animation_frame="Year",
                   )
fig.update_layout(title="MEAN FOOD CONSUMPTION IN AFRICA FROM 2004-2013")
fig.show()

FROM THE FIGURE ABOVE, WE ALSO SEE THAT THE TOP FOOD CONSUMING COUNTRY IN 2013 IS:
    1. EGYPT AT 3522KCAL/PERSON
    
OBSERVATION:
    1. EGYPT PRODUCES MORE AND EQUALLY CONSUMES MORE (AN IDEAL CASE).
    2. DJIBOUTI PRODUCES THE LEAST FOOD (79KT), YET CONSUMES ALOT OF FOOD(2136/KCAL/PERSON).
    3. NIGERIA PRODUCED THE MOST FOOD, YET CONSUMED FAR LESS THAN EXPECTED.
THIS POSES A RATHER INTERESTING QUESTION...
 

# WHY ARE SOME COUNTRIES PRODUCING MORE, YET CONSUMING LESS?

WITH THE AID OF A BOXPLOT SHOWING THE AVERAGE FOOD CONSUMED PER YEAR, WE CAN SELECT A YEAR WITHOUT OUTLIERS, SINCE WE KNOW THAT OUTLIERS DONT HELP MUCH IN ANALYSIS AS THEY GENERALLY DO NOT FOLLOW THE PATTERN.

In [42]:
trc = go.Box(
        y = supply.Value,
        x = supply.Year,
        boxpoints = 'suspectedoutliers',
        boxmean = True,
        notched=True,
)
data = [trc]
fig = go.Figure(data)
fig.update_layout(title="Food Consumption in all countries from 2004-2013")
iplot(fig)

THE YEAR 2013 WILL BE USED AS IT IS BY FAR THE MOST RECENT RECORD, AND IS ALSO FREE OF OUTLIERS THAT GENERALLY DO NOT FOLLOW THE PATTERN OF an ANALYSIS.

In [43]:
s_2013 = supply[supply.Year == 2013]
s_2013 = s_2013.rename({"Value":"Supply_value"},axis=1)
#s_2013.head()
p_2013 = prod[prod.Year == 2013]
p_2013 = p_2013.rename({"Value":"Prod_value"},axis=1)
#p_2013.head()
#df_2013=p_2013.join(other=s_2013)
df_2013 = pd.merge(p_2013,s_2013)
df_2013 = df_2013.set_index("Country")
#df_2013.head()

In [44]:
df_2013 = df_2013.join(other=CC)
df_2013 = df_2013.reset_index()
#df_2013

BY QUICKLY IMPORTING THE POPULATION OF THE THREE COUNTRIES UNDER ANALYSIS FOR THE YEAR 2013, WE SEE THAT:

In [45]:
DJ_pop_2013=pop[pop["Country Name"]=="Djibouti"]
DJ_pop_2013=pop.at[54,"2013"]
print("THE POPULATION OF DJIBOUTI IS: ",DJ_pop_2013)

E_pop_2013=pop[pop["Country Name"]=="Egypt"]
E_pop_2013=pop.at[65,"2013"]
print("THE POPULATION OF EGYPT IS: ",E_pop_2013)

N_pop_2013=pop[pop["Country Name"]=="Nigeria"]
N_pop_2013=pop.at[172,"2013"]
print("THE POPULATION OF NIGERIA IS: ",N_pop_2013)

THE POPULATION OF DJIBOUTI IS:  883293.0
THE POPULATION OF EGYPT IS:  88404640.0
THE POPULATION OF NIGERIA IS:  171765769.0


In [46]:
#A is the population of Nigeria
A = np.array([171765769.0]*10)
A = pd.DataFrame(A).rename({0:"Population"},axis=1)

#B is the population of Egypt
B = np.array([88404640.0]*10)
B = pd.DataFrame(B).rename({0:"Population"},axis=1)

#A is the population of Djibouti
C = np.array([883293.0]*10)
C = pd.DataFrame(C).rename({0:"Population"},axis=1)
#C

In [47]:
colors = ['blue',] * 2
#colors[1]="red"
x=['NIGERIA','EGYPT', "DJIBOUTI"]
y= [N_pop_2013, E_pop_2013, DJ_pop_2013]

fig = go.Figure(data=[go.Bar(x=x,
                        y=y,
                        marker_color=colors,
                        text=y,
                        textposition="auto",
                        
                            textsrc="")
                        ])
fig.update_layout(
    title_text='POPULATION OF EGYPT, DJIBOUTI & NIGERIA IN YR 2013',
    yaxis_type="log")
fig.show()

A QUICK LOOK AT THE POPULATION OF THE THREE COUNTRIES UNDER CONSIDERATION REVEALS THAT NIGERIA HAS THE HIGHEST POPULATION, FOLLOWED BY EGYPT AND THEN DJIBOUTI AT AN ALMOST INSIGNIFICANT VALUE!!!

In [48]:
p_Nig=prod[prod["Country"]=="Nigeria"].rename({"Value":"Prod_val"}, axis=1).reset_index()
s_Nig = supply[supply["Country"]=="Nigeria"].rename({"Value":"sup_val"}, axis=1).reset_index()
N_2013 = p_Nig.join([s_Nig["sup_val"],A])
#N_2013

In [49]:
p_Egypt = prod[prod["Country"]=="Egypt"].rename({"Value":"Prod_val"}, axis=1).reset_index()
s_Egypt = supply[supply["Country"]=="Egypt"].rename({"Value":"sup_val"}, axis=1).reset_index()
E_2013 = p_Egypt.join([s_Egypt["sup_val"],B])
#E_2013

In [50]:
p_Dj = prod[prod["Country"]=="Djibouti"].rename({"Value":"Prod_val"}, axis=1).reset_index()
s_Dj = supply[supply["Country"]=="Djibouti"].rename({"Value":"sup_val"}, axis=1).reset_index()
Dj_2013 = p_Dj.join([s_Dj["sup_val"],C])
#Dj_2013

In [55]:
last_df = pd.concat([N_2013,E_2013,Dj_2013])
last_df = last_df.drop(["index"],axis=1)
#last_df

In [52]:
long_df = last_df

fig = px.bar(long_df, x="Country", 
             y="Population", 
             color="Prod_val", 
             title="PLOT OF POPULATION VS COUNTRY WITH A VARITION IN PRODUCTION VALUE")
fig.update_layout( yaxis_type="log")
fig.show()

FROM THIS PLOT, WE CAN EASILY SEE THAT: 
    1. NIGERIA HAS THE HIGHEST POPULATION (REL. TO THE Y-AXIS), AND THE HIGHEST PRODUCTION (REL. TO THE COLOR SCALE). 
    2. EGYPT HAS A HIGH POPULATION (REL. TO THE Y-AXIS), AND A HIGH PRODUCTION (REL. TO THE COLOR SCALE). 
    3.DJIBOUTI HAS THE LOWEST POPULATION (REL. TO THE Y-AXIS), AND THE LOWEST PRODUCTION (REL. TO THE COLOR SCALE)

In [53]:
long_df = last_df

fig = px.bar(long_df,
             x="Country", 
             y="Population", 
             color="sup_val",
             title="PLOT OF POPULATION VS COUNTRY WITH A VARITION IN CONSUMTION VALUE")
fig.update_layout( yaxis_type="log")
fig.show()

FORM THE ABOVE CHARTS, WE CAN DRAW THE FOLLOWING INSIGHTS:
    1. NIGERIA HAS THE HIGHEST POPULATION AND PRODUCTION, BUT CONSUMES FAR LESS.
    2. EGYPT HAS AN HIGH POPULATION AND CONSUMES HIGH AMOUNT OF FOOD.
    3. DJIBOUTI HAS A LOW POPULATION, PRODUCES LESS, BUT CONSUMES FAR ABOVE WHAT IT PRODUCES

In [54]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Table(header=dict(values=["COUNTRY",'POPULATION', 'PRODUCTION(KT)', "CONSUMPTION(KCAL/P)"]),
                 cells=dict(values=[
                     ["NIGERIA" ,"EGYPT", "DJIBOUTI"], 
                     ["VERY HIGH", "HIGH", "VERY LOW"],
                     ["VERY HIGH", "HIGH", "VERY LOW"],
                     ["REL. LOW", "HIGH",  "REL. HIGH"]
                 ]))
                     ])
fig.update_layout(title="SUMMARY TABLE")
fig.show()

# IN CONCLUSION...

FROM THE ANALYSIS, THESE INSIGHTS HIGHLIGHTS TWO POSSIBLE REASONS CAUSING FOOD SHORTAGE IN AFRICA: 

A. POSSIBLE EXPORTATION ACTIVITIES AT THE DETRIMENT OF THE COUNTRY


SEEING THAT NIGERIA HAS THE HIGHEST POPULATION AND PRODUCES THE MOST FOOD, BUT YET CONSUMES WAY LESS, IT IS POSSIBLE THAT EXCESSIVE/ILLEGAL FOOD PRODUCTION OCCURS WITHIN THE COUNTRY, WHICH COULD EXPLAIN WHY COUNTRIES LIKE DJIBOUTI HAVING THE LEAST PRODUCTION HOWEVER CONSUMES FAR MORE THAN IT PRODUCES.

B. POSSIBLE MISAPPROPRIATION OF FOOD WITHIN THE COUNTRY

GIVEN THE HIGH PRODUCTION OF FOOD IN NIGERIA, AND ITS RELATIVELY LOW CONSUMPTION, IT IS LOGICAL TO INFER THAT THERE IS POSSIBLE MISAPPROPRIATION OF FOOD IN CERTAIN AREAS WHERE SOME STATES HAVE EXCESS FOOD AND OTHERS LACK ENOUGH.
SUCH A SCENARIO AS THIS LEADS TO WASTAGE THAT COULD LEAD TO SHORTAGE OF FOOD SUPPLY IN AFRICA.

# POSSIBLE SOLUTIONS

1. THE AUTHORITIES SHOULD BE KEEN ON THE ALLOCATION OF FOOD WITHIN THE COUNTRY TO PREVENT WASTE AND SCARCITY.
2. ILLEGAL/EXCESSIVE FOOD EXPORTATION SHOULD BE CURTAILED